In [ ]:
#Setup
import numpy as np
from datascience import *
from IPython.display import Image
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

def regression_line(r):
    x = np.random.normal(0, 1, 10000)
    z = np.random.normal(0, 1, 10000)
    y = r*x + (np.sqrt(1-r**2))*z
    plots.figure(figsize=(6, 6))
    plots.xlim(-4, 4)
    plots.ylim(-4, 4)
    plots.scatter(x, y)
    plots.plot([-4, 4], [-4*r,4*r], color='g', lw=2)
    if r >= 0:
        plots.plot([-4,4],[-4,4], lw=2, color='r')
    else:
        plots.plot([-4,4], [4,-4], lw=2, color='r')


# DSC 10 Discussion 9

Topics:
    - Correlation
        - Standard units
        - r constant
    - Regression line
    - The Method of Least Squares
        - Error Estimation
    
        

# Correlation

cor·re·la·tion    `kôrəˈlāSH(ə)n`    -noun
    
a mutual relationship or connection between two or more things.

"research showed a clear correlation between recession and levels of property crime"


## Standard units

Last time we talked about how standard deviation can be made to compare across data types. 

The way we accomplish this is through a method called standard units


### Question 1

Say we take a set of numbers and subtract the mean of that set from every number. What is the significance of this number? Is this a Standard Unit?

*Write your answer and reasoning here*

### Question 2

What will it take to make this a Standard Unit?

*Write your answer and reasoning here*

### Question 3

Write a function to calculte standard units

In [ ]:
def standard_units(any_numbers):
    "Convert any array of numbers to standard units."
    return (any_numbers - np.mean(any_numbers))/np.std(any_numbers)  

## r constant

A use of standard units is not only to compare across datasets but to correlate attributes. One measure of correlation is that of r, the correlation coefficient. This value is an indication of two things. One how significantly accurate is your data. Two how good will your pediction be.

### Question 4

If you take the product of two standard units you end up with a measure of how far the data is from the regression line. 

From this knowledge describe a way to calculate the value of how close to the regression line the data is as a whole

*Write your answer and reasoning here*

### Question 5

This value is the same as the correlation coefficient,r.

Using the awnser above ceate a function returns the value r

In [ ]:
def Calc_r(x,y):
    t = Table().with_columns('x', x,'y', y)
    t_su = t.with_columns('x (standard units)', standard_units(x),'y (standard units)', standard_units(y))
    t_product = t_su.with_column('product of standard units', t_su.column(2) * t_su.column(3))
    return np.mean(t_product.column(4))

In [ ]:
x = np.arange(1, 7, 1)
y = make_array(2, 3, 1, 5, 2, 7)
print(Calc_r(x,y))

# Regression Line

In [ ]:
Image("Regression_Line_5_0.png")

In [ ]:
regression_line(0.95)

In regression, we use the value of one variable (which we will call $x$) to predict the value of another (which we will call $y$). When the variables $x$ and $y$ are measured in standard units, the regression line for predicting $y$ based on $x$ has slope $r$ and passes through the origin. Thus the equation of the regression line can be written as:

$$
\mbox{estimate of }y ~=~ r \cdot x ~~~
\mbox{when both variables are measured in standard units}
$$
 
In the original units of the data, this becomes 

$\frac{estimate\ of\ y − average\ of\ y}{SD\ of\ y} = r*\frac{the\ given\ x − average\ of\ x}{SD of x}$

The slope and intercept of the regression line in original units can be derived from the diagram above. 

$$
\mathbf{\mbox{slope of the regression line}} ~=~ r \cdot
\frac{\mbox{SD of }y}{\mbox{SD of }x}
$$

$$
\mathbf{\mbox{intercept of the regression line}} ~=~
\mbox{average of }y ~-~ \mbox{slope} \cdot \mbox{average of }x
$$

In [ ]:

def standard_units(any_numbers):
    "Convert any array of numbers to standard units."
    return (any_numbers - np.mean(any_numbers))/np.std(any_numbers)  

def correlation(t, x, y):
    return np.mean(standard_units(t.column(x))*standard_units(t.column(y)))

def slope(table, x, y):
    r = correlation(table, x, y)
    return r * np.std(table.column(y))/np.std(table.column(x))

def intercept(table, x, y):
    a = slope(table, x, y)
    return np.mean(table.column(y)) - a * np.mean(table.column(x))

def fit(table, x, y):
    """Return the height of the regression line at each x value."""
    a = slope(table, x, y)
    b = intercept(table, x, y)
    return a * table.column(x) + b
sample = [[131, 14431], [231, 20558], [392, 40935], [157, 23524]]
def lw_errors(slope, intercept):
    little_women.scatter('Periods', 'Characters')
    xlims = np.array([50, 450])
    plots.plot(xlims, slope * xlims + intercept, lw=2)
    for x, y in sample:
        plots.plot([x, x], [y, slope * x + intercept], color='r', lw=2)

# The Method of Least Squares #

This dataset that has one row for every chapter of the novel "Little Women." 

In [ ]:
little_women = Table.read_table('little_women.csv')
little_women = little_women.move_to_start('Periods')
little_women.show(3)

In [ ]:
little_women.scatter('Periods', 'Characters')

In [ ]:
correlation(little_women, 'Periods', 'Characters')

## Error Estimation

To understand what error we get we frist need to define what it means to have the best line.

In [ ]:
lw_with_predictions = little_women.with_column('Linear Prediction', fit(little_women, 'Periods', 'Characters'))
lw_with_predictions.scatter('Periods')

In [ ]:
actual = lw_with_predictions.column('Characters')
predicted = lw_with_predictions.column('Linear Prediction')
errors = actual - predicted

In [ ]:
lw_with_predictions.with_column('Error', errors)

We can use slope and intercept to calculate the slope and intercept of the fitted line. The graph below shows the line (in light blue). The errors corresponding to four of the points are shown in red. There is nothing special about those four points. They were just chosen for clarity of the display. The function lw_errors takes a slope and an intercept (in that order) as its arguments and draws the figure.

In [ ]:
lw_reg_slope = slope(little_women, 'Periods', 'Characters')
lw_reg_intercept = intercept(little_women, 'Periods', 'Characters')

In [ ]:
print('Slope of Regression Line:    ', np.round(lw_reg_slope), 'characters per period')
print('Intercept of Regression Line:', np.round(lw_reg_intercept), 'characters')
lw_errors(lw_reg_slope, lw_reg_intercept)

In [ ]:
lw_errors(50, 10000)

In [ ]:
lw_errors(-100, 50000)

### Question 6

How do we go about quantifying how `good` our line is?

*Write your answer and reasoning here*